### Alpha Vantage Income Statement Data Request

https://www.alphavantage.co/

In [4]:
# Import dependancies
import pandas as pd
import requests
import json
import sys
import time
import csv
import os

# Import the API key.
from config2 import stocks_p_api_key

In [5]:
IPO_df = pd.read_csv('raw_dataset/IPO_Listing_2019_2020.csv')
IPO_df.head()

,IPO_DATE,SYMBOL,NAME,IPO_PRICE,CURRENT,RETURN
0,2020-12-29,MRM,Medirom Healthcare,15.0,10.30,-0.3133
1,2020-12-24,VTAQ,Ventoux CCM Acquisition,10.0,9.89,-0.0110
2,2020-12-23,IKT,Inhibikase Therapeutics,10.0,6.21,-0.3790
3,2020-12-23,GBS,"GBS, Inc.",17.0,6.81,-0.5994
4,2020-12-23,HCAR,Healthcare Services Acquisition,10.0,9.85,-0.0155


In [6]:
IPO_df.shape

(710, 6)

In [7]:
IPO_df.dtypes

IPO_DATE      object
SYMBOL        object
NAME          object
IPO_PRICE    float64
CURRENT      float64
RETURN       float64
dtype: object

In [8]:
IPO_df['IPO_DATE'] = pd.to_datetime(IPO_df['IPO_DATE'], format = '%Y-%m-%d')

In [9]:
IPO_df.dtypes

IPO_DATE     datetime64[ns]
SYMBOL               object
NAME                 object
IPO_PRICE           float64
CURRENT             float64
RETURN              float64
dtype: object

In [10]:
IPO_df.head()

,IPO_DATE,SYMBOL,NAME,IPO_PRICE,CURRENT,RETURN
0,2020-12-29,MRM,Medirom Healthcare,15.0,10.30,-0.3133
1,2020-12-24,VTAQ,Ventoux CCM Acquisition,10.0,9.89,-0.0110
2,2020-12-23,IKT,Inhibikase Therapeutics,10.0,6.21,-0.3790
3,2020-12-23,GBS,"GBS, Inc.",17.0,6.81,-0.5994
4,2020-12-23,HCAR,Healthcare Services Acquisition,10.0,9.85,-0.0155


In [11]:
# Create a list of IPO symbols from the IPO_df
IPO_list = IPO_df['SYMBOL'].tolist()
IPO_list

['MRM',
 'VTAQ',
 'IKT',
 'GBS',
 'HCAR',
 'CFIV',
 'VII',
 'VHAQ',
 'ACKIT',
 'SVOK',
 'GFX',
 'MASS',
 'DUNE',
 'MTAC',
 'IIII',
 'MDWT',
 'SCOA',
 'COOL',
 'VIRI',
 'WISH',
 'UPST',
 'CCV',
 'SCPS',
 'BCAB',
 'WNW',
 'KINZ',
 'MRAC',
 'OCG',
 'BLUW',
 'ATA',
 'ABCL',
 'NBTX',
 'FDMT',
 'DWIN',
 'ROCC',
 'GLAQ',
 'MOTV',
 'EDTX',
 'SNRH',
 'GHVI',
 'TVAC',
 'CBAH',
 'VVOS',
 'CERT',
 'RMGB',
 'HYFM',
 'ABNB',
 'CTAQ',
 'PCPC',
 'NEBC',
 'DASH',
 'HMCO',
 'PUBM',
 'ALTU',
 'AI',
 'FLAC',
 'CND',
 'DDMX',
 'MUDS',
 'RAAC',
 'YQ',
 'SBTX',
 'SEER',
 'SGTX',
 'FTCV',
 'PTIC',
 'FPAC',
 'HTPA',
 'KNTE',
 'LOKB',
 'DCBO',
 'SPFR',
 'RSVA',
 'TACA',
 'CAP',
 'SGAM',
 'FRX',
 'SPRQ',
 'HFEN',
 'OZON',
 'SV',
 'VMAR',
 'GNPK',
 'VCVC',
 'TINV',
 'BREZ',
 'LSAQ',
 'ARBG',
 'MRVI',
 'CAS',
 'SHC',
 'KWAC',
 'OCA',
 'IIAC',
 'NGMS',
 'LNFA',
 'RTPZ',
 'TLS',
 'YSG',
 'OLMA',
 'JYAC',
 'CHFW',
 'PHIC',
 'DGNS',
 'PIPP',
 'OTRA',
 'ZNTE',
 'HAAC',
 'CFAC',
 'BWAC',
 'DMYI',
 'STIC',
 'NOAC',
 'TSI

In [12]:
# Check length of IPO_list
len(IPO_list)

710

### Request Income Statement from Alpha Vantage

Create a loop to retreive Income Statement for each symbol in the IPO_list

In [30]:
# Define url

base_income_statement_url = "https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol="

KEY_ERROR_List = []

Q_fiscalDateEnding_List = []
Q_reportedCurrency_List = []
Q_grossProfit_List = []
Q_totalRevenue_List = []
Q_costOfRevenue_List = []
Q_costofGoodsAndServicesSold_List = []
Q_operatingIncome_List = []
Q_sellingGeneralAndAdministrative_List = []
Q_researchAndDevelopment_List = []
Q_operatingExpenses_List = []
Q_investmentIncomeNet_List = []
Q_netInterestIncome_List = []
Q_interestIncome_List = []
Q_interestExpense_List = []
Q_nonInterestIncome_List = []
Q_otherNonOperatingIncome_List = []
Q_depreciation_List = []
Q_depreciationAndAmortization_List = []
Q_incomeBeforeTax_List = []
Q_incomeTaxExpense_List = []
Q_interestAndDebtExpense_List = []
Q_netIncomeFromContinuingOperations_List = []
Q_comprehensiveIncomeNetOfTax_List = []
Q_ebit_List = []
Q_ebitda_List = []
Q_netIncome_List = []


# Create For loop to fetch responses from Alpha Vantage

symbol_parameter = 'APLT'

full_url = f'{base_income_statement_url}{symbol_parameter}&apikey={stocks_p_api_key}'

# Make an API call to the API and get the JSON object
response = requests.get(full_url).json()

for i in range(len(response['quarterlyReports'])):

    Q_fiscalDateEnding_response = response['quarterlyReports'][i]['fiscalDateEnding']
    Q_reportedCurrency_response = response['quarterlyReports'][i]['reportedCurrency']
    Q_grossProfit_response = response['quarterlyReports'][i]['grossProfit']
    Q_totalRevenue_response = response['quarterlyReports'][i]['totalRevenue']
    Q_costOfRevenue_response = response['quarterlyReports'][i]['costOfRevenue']
    Q_costofGoodsAndServicesSold_response = response['quarterlyReports'][i]['costofGoodsAndServicesSold']
    Q_operatingIncome_response = response['quarterlyReports'][i]['operatingIncome']
    Q_sellingGeneralAndAdministrative_response = response['quarterlyReports'][i]['sellingGeneralAndAdministrative']
    Q_researchAndDevelopment_response = response['quarterlyReports'][i]['researchAndDevelopment']
    Q_operatingExpenses_response = response['quarterlyReports'][i]['operatingExpenses']
    Q_investmentIncomeNet_response = response['quarterlyReports'][i]['investmentIncomeNet']
    Q_netInterestIncome_response = response['quarterlyReports'][i]['netInterestIncome']
    Q_interestIncome_response = response['quarterlyReports'][i]['interestIncome']
    Q_interestExpense_response = response['quarterlyReports'][i]['interestExpense']
    Q_nonInterestIncome_response = response['quarterlyReports'][i]['nonInterestIncome']
    Q_otherNonOperatingIncome_response = response['quarterlyReports'][i]['otherNonOperatingIncome']
    Q_depreciation_response = response['quarterlyReports'][i]['depreciation']
    Q_depreciationAndAmortization_response = response['quarterlyReports'][i]['depreciationAndAmortization']
    Q_incomeBeforeTax_response = response['quarterlyReports'][i]['incomeBeforeTax']
    Q_incomeTaxExpense_response = response['quarterlyReports'][i]['incomeTaxExpense']
    Q_interestAndDebtExpense_response = response['quarterlyReports'][i]['interestAndDebtExpense']
    Q_netIncomeFromContinuingOperations_response = response['quarterlyReports'][i]['netIncomeFromContinuingOperations']
    Q_comprehensiveIncomeNetOfTax_response = response['quarterlyReports'][i]['comprehensiveIncomeNetOfTax']
    Q_ebit_response = response['quarterlyReports'][i]['ebit']
    Q_ebitda_response = response['quarterlyReports'][i]['ebitda']
    Q_netIncome_response = response['quarterlyReports'][i]['netIncome']

    Q_fiscalDateEnding_List.append(Q_fiscalDateEnding_response)
    Q_reportedCurrency_List.append(Q_reportedCurrency_response)
    Q_grossProfit_List.append(Q_grossProfit_response)
    Q_totalRevenue_List.append(Q_totalRevenue_response)
    Q_costOfRevenue_List.append(Q_costOfRevenue_response)
    Q_costofGoodsAndServicesSold_List.append(Q_costofGoodsAndServicesSold_response)
    Q_operatingIncome_List.append(Q_operatingIncome_response)
    Q_sellingGeneralAndAdministrative_List.append(Q_sellingGeneralAndAdministrative_response)
    Q_researchAndDevelopment_List.append(Q_researchAndDevelopment_response)
    Q_operatingExpenses_List.append(Q_operatingExpenses_response)
    Q_investmentIncomeNet_List.append(Q_investmentIncomeNet_response)
    Q_netInterestIncome_List.append(Q_netInterestIncome_response)
    Q_interestIncome_List.append(Q_interestIncome_response)
    Q_interestExpense_List.append(Q_interestExpense_response)
    Q_nonInterestIncome_List.append(Q_nonInterestIncome_response)
    Q_otherNonOperatingIncome_List.append(Q_otherNonOperatingIncome_response)
    Q_depreciation_List.append(Q_depreciation_response)
    Q_depreciationAndAmortization_List.append(Q_depreciationAndAmortization_response)
    Q_incomeBeforeTax_List.append(Q_incomeBeforeTax_response)
    Q_incomeTaxExpense_List.append(Q_incomeTaxExpense_response)
    Q_interestAndDebtExpense_List.append(Q_interestAndDebtExpense_response)
    Q_netIncomeFromContinuingOperations_List.append(Q_netIncomeFromContinuingOperations_response)
    Q_comprehensiveIncomeNetOfTax_List.append(Q_comprehensiveIncomeNetOfTax_response)
    Q_ebit_List.append(Q_ebit_response)
    Q_ebitda_List.append(Q_ebitda_response)
    Q_netIncome_List.append(Q_netIncome_response)



In [31]:
# Transform each list into a series

Q_fiscalDateEnding_Series = pd.Series(Q_fiscalDateEnding_List)
Q_reportedCurrency_Series = pd.Series(Q_reportedCurrency_List)
Q_grossProfit_Series = pd.Series(Q_grossProfit_List)
Q_totalRevenue_Series = pd.Series(Q_totalRevenue_List)
Q_costOfRevenue_Series = pd.Series(Q_costOfRevenue_List)
Q_costofGoodsAndServicesSold_Series = pd.Series(Q_costofGoodsAndServicesSold_List)
Q_operatingIncome_Series = pd.Series(Q_operatingIncome_List)
Q_sellingGeneralAndAdministrative_Series = pd.Series(Q_sellingGeneralAndAdministrative_List)
Q_researchAndDevelopment_Series = pd.Series(Q_researchAndDevelopment_List)
Q_operatingExpenses_Series = pd.Series(Q_operatingExpenses_List)
Q_investmentIncomeNet_Series = pd.Series(Q_investmentIncomeNet_List)
Q_netInterestIncome_Series = pd.Series(Q_netInterestIncome_List)
Q_interestIncome_Series = pd.Series(Q_interestIncome_List)
Q_interestExpense_Series = pd.Series(Q_interestExpense_List)
Q_nonInterestIncome_Series = pd.Series(Q_nonInterestIncome_List)
Q_otherNonOperatingIncome_Series = pd.Series(Q_otherNonOperatingIncome_List)
Q_depreciation_Series = pd.Series(Q_depreciation_List)
Q_depreciationAndAmortization_Series = pd.Series(Q_depreciationAndAmortization_List)
Q_incomeBeforeTax_Series = pd.Series(Q_incomeBeforeTax_List)
Q_incomeTaxExpense_Series = pd.Series(Q_incomeTaxExpense_List)
Q_interestAndDebtExpense_Series = pd.Series(Q_interestAndDebtExpense_List)
Q_netIncomeFromContinuingOperations_Series = pd.Series(Q_netIncomeFromContinuingOperations_List)
Q_comprehensiveIncomeNetOfTax_Series = pd.Series(Q_comprehensiveIncomeNetOfTax_List)
Q_ebit_Series = pd.Series(Q_ebit_List)
Q_ebitda_Series = pd.Series(Q_ebitda_List)
Q_netIncome_Series = pd.Series(Q_netIncome_List)


In [39]:
frame = {
    'SYMBOL':symbol_parameter,
    'fiscalDateEnding' : Q_fiscalDateEnding_Series,
    'reportedCurrency' : Q_reportedCurrency_Series,
    'grossProfit' : Q_grossProfit_Series,
    'totalRevenue' : Q_totalRevenue_Series,
    'costOfRevenue' : Q_costOfRevenue_Series,
    'costofGoodsAndServicesSold' : Q_costofGoodsAndServicesSold_Series,
    'operatingIncome' : Q_operatingIncome_Series,
    'sellingGeneralAndAdministrative' : Q_sellingGeneralAndAdministrative_Series,
    'researchAndDevelopment' : Q_researchAndDevelopment_Series,
    'operatingExpenses' : Q_operatingExpenses_Series,
    'investmentIncomeNet' : Q_investmentIncomeNet_Series,
    'netInterestIncome' : Q_netInterestIncome_Series,
    'interestIncome' : Q_interestIncome_Series,
    'interestExpense' : Q_interestExpense_Series,
    'nonInterestIncome' : Q_nonInterestIncome_Series,
    'otherNonOperatingIncome' : Q_otherNonOperatingIncome_Series,
    'depreciation' : Q_depreciation_Series,
    'depreciationAndAmortization' : Q_depreciationAndAmortization_Series,
    'incomeBeforeTax' : Q_incomeBeforeTax_Series,
    'incomeTaxExpense' : Q_incomeTaxExpense_Series,
    'interestAndDebtExpense' : Q_interestAndDebtExpense_Series,
    'netIncomeFromContinuingOperations' : Q_netIncomeFromContinuingOperations_Series,
    'comprehensiveIncomeNetOfTax' : Q_comprehensiveIncomeNetOfTax_Series,
    'ebit' : Q_ebit_Series,
    'ebitda' : Q_ebitda_Series,
    'netIncome' : Q_netIncome_Series
}

In [40]:
q_income_statement_df = pd.DataFrame(frame)
q_income_statement_df

,SYMBOL,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
0,APLT,2020-12-31,USD,None,0,None,None,-23747000,9933000,13814000,...,None,None,-23688000,None,None,-23688000,None,-23648000,None,-23689000
1,APLT,2020-09-30,USD,None,0,None,None,-29965000,10020000,19945000,...,None,None,-29844000,None,None,-29844000,None,-29870000,None,-29844000
2,APLT,2020-06-30,USD,None,0,None,None,-28279000,7521000,20758000,...,None,None,-28058000,None,None,-28058000,None,-28186000,None,-28059000
3,APLT,2020-03-31,USD,None,0,None,None,-12467000,5196000,7271000,...,None,None,-12369000,None,None,-12369000,None,-12374000,None,-12369000
4,APLT,2019-12-31,USD,None,0,None,None,-17669000,3901000,13768000,...,None,None,-17633000,None,None,-17633000,None,-17669000,None,-17633000
5,APLT,2019-09-30,USD,None,0,None,None,-10747000,3294000,7453000,...,None,None,-10713000,None,None,-10713000,None,-10747000,None,-10713000
6,APLT,2019-06-30,USD,None,0,None,None,-8437000,4183000,4254000,...,None,None,-8437000,None,None,-8437000,None,-8437000,None,-8437000
7,APLT,2019-03-31,USD,None,0,None,None,-8729000,1855000,6874000,...,None,None,-8730000,None,None,-8730000,None,-8729000,None,-8730000
8,APLT,2018-09-30,USD,None,None,None,None,-3324000,598000,2726000,...,None,None,-4281000,None,None,-4281000,None,-3324000,None,-4281000
9,APLT,2018-06-30,USD,None,None,None,None,-2337000,400000,1937000,...,None,None,-3185000,None,None,-3185000,None,-2337000,None,-3185000


In [41]:
# Exporting test income statement dataframe to csv file
q_income_statement_df.to_csv('raw_dataset/test/test_income_statement.csv')